<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/python_/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import load_sample_image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array

# 1.  모델 로드
model = InceptionV3(weights='imagenet')

# 2. sklearn에서 제공하는 샘플 이미지 로드
# 사용 가능한 샘플 이미지: 'china.jpg', 'flower.jpg'
image = load_sample_image('china.jpg')  # 또는 'flower.jpg'
image = img_to_array(image)

# 3. 이미지 전처리
# InceptionV3 모델에 입력할 수 있도록 이미지 크기를 299x299로 조정하고, 전처리
image = np.resize(image, (299, 299, 3))
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# 4. 예측 수행
predictions = model.predict(image)

# 5. 예측 결과 디코딩 및 출력
decoded_predictions = decode_predictions(predictions, top=3)[0]
print("Predicted:", decoded_predictions)

# 출력된 결과를 해석
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i+1}: {label} ({score:.2f})")


96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
Predicted: [('n03891251', 'park_bench', 0.15180102), ('n03777754', 'modem', 0.049145788), ('n03223299', 'doormat', 0.04304461)]
1: park_bench (0.15)
2: modem (0.05)
3: doormat (0.04)


1. park_bench (0.15):

  모델은 이미지가 "park bench" (공원 벤치)일 가능성을 약 15.18%로 예측했습니다.
  이 예측이 가장 높은 확률로, 모델이 이미지에서 공원 벤치와 관련된 특징을 인식했을 가능성이 큽니다.

2. modem (0.05):

  모델은 이미지가 "modem" (모뎀)일 가능성을 약 4.91%로 예측했습니다.
  이는 이미지에서 모뎀과 관련된 특징이 일부 인식되었음을 시사합니다.

3. doormat (0.04):

  모델은 이미지가 "doormat" (현관 매트)일 가능성을 약 4.30%로 예측했습니다.
  이미지에서 현관 매트와 유사한 특징이 발견되었을 가능성이 있습니다.

In [3]:
import numpy as np
from sklearn.datasets import load_sample_image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 1. InceptionV3 모델 로드 및 미세 조정 설정
base_model = InceptionV3(weights='imagenet', include_top=False)  # 최상위 레이어 제거

# 2. 레이어 추가 및 모델 확장
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 추가
predictions = Dense(1000, activation='softmax')(x)  # ImageNet 데이터셋의 클래스 수

# 새 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 3. 일부 레이어 동결 해제 (Fine-tuning)
for layer in base_model.layers[-50:]:  # 마지막 50개 레이어는 학습 가능하도록 설정
    layer.trainable = True

# 4. 모델 컴파일 (하이퍼파라미터 튜닝)
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# 5. 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 데이터의 20%를 검증 데이터로 사용
)

# 6. 샘플 이미지 로드 및 전처리 (여기서는 데이터 증강 예시로 사용)
# 실제로는 여러 이미지를 사용해야 하므로, 아래는 데이터 로딩의 예시입니다.
image = load_sample_image('china.jpg')  # 예제 이미지
image = img_to_array(image)
image = np.resize(image, (299, 299, 3))  # InceptionV3 입력 크기에 맞추기
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# 7. 가상 데이터를 사용한 데이터 증강
# 실제로는 여러 샘플을 로드하고 데이터셋을 구축해야 합니다.
train_data, val_data, train_labels, val_labels = train_test_split([image]*100, to_categorical([1]*100, 1000), test_size=0.2)

train_data = np.squeeze(train_data, axis=1) # Remove the extra dimension from train_data
val_data = np.squeeze(val_data, axis=1) # Remove the extra dimension from val_data

train_generator = datagen.flow(
    np.array(train_data),
    np.array(train_labels),
    batch_size=32,
    subset='training'
)

val_generator = datagen.flow(
    np.array(val_data),
    np.array(val_labels),
    batch_size=32,
    subset='validation'
)

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 8. 모델 학습
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping]
)

# 9. 모델 평가 (검증 데이터에서)
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# 10. 예측 수행 (여기서는 샘플 이미지로 예시)
y_pred = model.predict(image)
decoded_predictions = decode_predictions(y_pred, top=3)[0]
print("Predicted:", decoded_predictions)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 95s 12s/step - accuracy: 0.0000e+00 - loss: 6.6315 - val_accuracy: 0.0000e+00 - val_loss: 6.5073
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 312ms/step - accuracy: 0.0000e+00 - loss: 6.5239 - val_accuracy: 0.0000e+00 - val_loss: 6.2298
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 0.0312 - loss: 6.3609 - val_accuracy: 0.0000e+00 - val_loss: 6.2129
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 302ms/step - accuracy: 0.0312 - loss: 6.2156 - val_accuracy: 1.0000 - val_loss: 5.7569
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 301ms/step - accuracy: 0.2396 - loss: 5.9154 - val_accuracy: 1.0000 - val_loss: 5.7286
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 310ms/step - accuracy: 0.4167 - loss: 5.7153 - val_accuracy: 1.0000 - val_loss: 5.4394
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - accuracy: 0.4062 - loss: 5.6884 - val_accuracy: 0.7500 - val_loss: 5.4589
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 296ms/step - accuracy: 0.6354 - loss: 5.4693 - val_accuracy: 1.0000 - v

모델이 성공적으로 학습되었고, 검증 데이터에서 매우 높은 정확도와 낮은 손실 값을 달성한 것을 볼 수 있습니다. 마지막 예측 결과에서 확률이 높은 "goldfish" 클래스를 예측한 것을 보면, 모델이 입력 이미지를 잘 학습하고 예측하는 것을 확인할 수 있습니다.

다른 데이터셋을 사용하여 InceptionV3 모델을 평가하겠습니다. 여기서는 CIFAR-10 데이터셋을 사용하여 모델의 성능을 검증하는 방법을 예로 들겠습니다. CIFAR-10은 10개의 클래스(예: 비행기, 자동차, 새, 고양이 등)로 구성된 6만 개의 32x32 크기의 컬러 이미지로 이루어져 있습니다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# 1. CIFAR-10 데이터셋 로드 및 전처리
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# InceptionV3 모델에 맞게 이미지 크기를 75x75로 조정 (InceptionV3의 입력 크기)
x_train = tf.image.resize(x_train, (75, 75))
x_test = tf.image.resize(x_test, (75, 75))

# 데이터 전처리 (픽셀 값 범위 조정 및 전처리 함수 적용)
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# 레이블을 One-hot 인코딩
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 2. InceptionV3 모델 로드 및 미세 조정 설정
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))

# 3. 레이어 추가 및 모델 확장
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)  # 배치 정규화 추가
x = Dropout(0.7)(x)  # 드롭아웃 비율 증가
predictions = Dense(10, activation='softmax')(x)  # CIFAR-10 데이터셋의 클래스 수

# 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 4. 일부 레이어 동결 해제 (Fine-tuning)
for layer in base_model.layers[-50:]:
    layer.trainable = True

# 5. 모델 컴파일 (학습률 조정)
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# 6. 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 데이터 증강 적용
train_generator = datagen.flow(x_train, y_train, batch_size=32)

# 7. 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 8. 모델 학습
model.fit(
    train_generator,
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)

# 9. 모델 평가
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# 10. 예측 수행
y_pred = model.predict(x_test)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step
Epoch 1/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 184s 77ms/step - accuracy: 0.1285 - loss: 3.8264 - val_accuracy: 0.3270 - val_loss: 2.0007
Epoch 2/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 87s 56ms/step - accuracy: 0.2149 - loss: 2.8566 - val_accuracy: 0.4520 - val_loss: 1.6613
Epoch 3/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 87s 55ms/step - accuracy: 0.3044 - loss: 2.3763 - val_accuracy: 0.5360 - val_loss: 1.3981
Epoch 4/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 88s 56ms/step - accuracy: 0.3720 - loss: 2.0786 - val_accuracy: 0.5969 - val_loss: 1.2085
Epoch 5/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - accuracy: 0.4369 - loss: 1.8888 - val_accuracy: 0.6399 - val_loss: 1.0945
Epoch 6/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 87s 55ms/step - accuracy: 0.4834 - loss: 1.7354 - val_accuracy: 0.6753 - val_loss: 0.9797
Epoch 7/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 88s 56ms/step - accuracy: 0.5168 - loss: 1.6442 - val_accuracy: 0.7066 - val_loss: 0.8823
Epoch 8/50
1563/1563 